In [1]:
# Import necessary modules
from jupyter_dash import JupyterDash
import dash_leaflet as dl
from dash import dcc, html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output
import base64
import pandas as pd
import numpy as np
from animal_shelter import AnimalShelter

# Data Manipulation / Model
shelter = AnimalShelter()

# Query data from the shelter (retrieving all records as an example)
df = pd.DataFrame.from_records(shelter.read({}))

# Remove `_id` if present (MongoDB ID handling)
if '_id' in df.columns:
    df = df.drop(columns=['_id'])

# Flatten `rescue_type` if it contains arrays
df['rescue_type'] = df['rescue_type'].apply(
    lambda x: ", ".join(x) if isinstance(x, list) else x
)

# Define the possible rescue types from the provided data
RESCUE_TYPE_MAP = {
    "Water Rescue": ["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"],
    "Mountain or Wilderness Rescue": ["German Shepherd", "Alaskan Malamute", "Old English Sheepdog", "Siberian Husky", "Rottweiler"],
    "Disaster or Individual Tracking": ["Doberman Pinscher", "German Shepherd", "Golden Retriever", "Bloodhound", "Rottweiler"]
}

# Create Dash app
app = JupyterDash(__name__)

# Load Grazioso Salvare logo as Base64
image_filename = 'Grazioso Salvare Logo.png'  # Replace with actual logo path
encoded_image = base64.b64encode(open(image_filename, 'rb').read()).decode()

# Define layout of the dashboard
app.layout = html.Div([
    html.Div(id='hidden-div', style={'display': 'none'}),
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard'))),
    html.H3('Unique Identifier: aacuser'),
    html.Hr(),

    # Display the logo
    html.Div(
        html.A(
            html.Img(
                src=f'data:image/png;base64,{encoded_image}',
                style={"width": "200px", "height": "auto"}
            ),
            href="https://www.snhu.edu", target="_blank"
        ),
        style={"textAlign": "center"}
    ),

    # Interactive filters for Rescue Type
    html.Div([
        html.Label("Filter by Rescue Type"),
        dcc.Dropdown(
            id='filter-rescue-type',
            options=[
                {"label": "Water Rescue", "value": "Water Rescue"},
                {"label": "Mountain or Wilderness Rescue", "value": "Mountain or Wilderness Rescue"},
                {"label": "Disaster or Individual Tracking", "value": "Disaster or Individual Tracking"},
                {"label": "Reset", "value": "Reset"}
            ],
            value="Reset",
            clearable=False
        )
    ], style={"width": "50%", "margin": "10px auto", "textAlign": "center"}),

    # Create a data table
    dash_table.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        style_table={'overflowX': 'auto'},
        page_size=10,
        sort_action="native",
        filter_action="native",
        row_selectable="single",
        selected_rows=[0]  # Pre-select the first row
    ),

    html.Br(),
    html.Hr(),

    html.Div([
    # Map to dynamically show data from the table
        html.Div([
            html.Div(id='map-id',
            style={"width": "100%", "height": "500px"}), 
        ], style={'flex':'1', 'margin-right':'10px'}),

    # Pie chart for displaying breed distribution
        html.Div([
            dcc.Graph(id='pie-chart', style={'height':'500px'}),
        ], style={'flex':'1','margin-left':'10px'}),
        ], style={'display': 'flex', 'flex-direction': 'row'}),
]) 

# Callback to filter data based on user selection
@app.callback(
    Output('datatable-id', 'data'),
    [Input('filter-rescue-type', 'value')]
)
def filter_data(rescue_type):
    filtered_df = df
    if rescue_type != "Reset":
        filtered_df = filtered_df[filtered_df['rescue_type'] == rescue_type]
    return filtered_df.to_dict('records')

# Callback to update the pie chart based on the filters
@app.callback(
    Output('pie-chart', 'figure'),
    [Input('filter-rescue-type', 'value')]
)
def update_pie_chart(rescue_type):
    filtered_df = df
    if rescue_type != "Reset":
        filtered_df = filtered_df[filtered_df['rescue_type'] == rescue_type]
    
    #Calculate breed counts.
    breed_counts = filtered_df['breed'].value_counts()
    
    #Limit to top 10 breeds and combine others into "Other"
    top_n = 10
    if len(breed_counts) > top_n:
        top_breeds = breed_counts[:top_n]
        other_count = breed_counts[top_n:].sum()
        breed_counts = top_breeds.append(pd.Series({'Other': other_count}))
        
    
    fig = px.pie(
        breed_counts, 
        names=breed_counts.index, 
        values=breed_counts.values, 
        title='Breed Distribution')
    return fig

# Callback to update the map based on selected row
@app.callback(
    Output('map-id', 'children'),
    [Input('datatable-id', 'derived_virtual_data'),
     Input('datatable-id', 'derived_virtual_selected_rows')]
)
def update_map(viewData, index):
    if viewData is None or index is None or len(index) == 0:
        return dl.Map(style={'width': '1000px', 'height': '500px'}, center=[0, 0], zoom=1, children=[dl.TileLayer()])

    # Extract the selected row's location data
    dff = pd.DataFrame.from_dict(viewData)
    row = index[0]

    lat = dff.iloc[row]['location_lat']
    lon = dff.iloc[row]['location_long']

    # Render the map with a marker for the selected row
    return dl.Map(
        style={'width': '1000px', 'height': '500px'},
        center=[lat, lon],
        zoom=15,
        children=[
            dl.TileLayer(),
            dl.Marker(
                position=[lat, lon],
                children=[
                    dl.Tooltip(dff.iloc[row]['breed']),
                    dl.Popup(html.H1(dff.iloc[row]['name']))
                ]
            )
        ]
    )

# Run the server
app.run_server(debug=True)

Dash app running on http://127.0.0.1:18348/


/tmp/ipykernel_341564/1920546493.py:138: FutureWarning:

The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

